# GraphRAG Core: Climate Intelligence Tutorial

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nunezmatias/grafoRag/blob/main/examples/Tutorial_GraphRAG.ipynb)

Welcome to the GraphRAG Core interactive research tool. This system bridges the gap between unstructured scientific literature and structured causal knowledge.


## 1. Installation


In [ ]:
!pip install git+https://github.com/nunezmatias/grafoRag.git
!pip install -q -U google-genai

import os
from graphrag_core import GraphRAGEngine
print('Libraries Installed & Loaded')


## 2. Initialize the Engine


In [ ]:
engine = GraphRAGEngine()
# System will download data if missing


## 3. Run a Deep Research Query


In [ ]:
# Define your research question
query = "cascading risks of extreme heat and urban floods"

# Execute the Search
results = engine.search(
    query=query, 
    top_k=3, 
    context_k=4, 
    hops=2
)

print("--- Research Stats ---")
print(f"Primary Sources: {results["stats"]["primary"]}")
print(f"Context Expansion: {results["stats"]["context"]}")
print(f"Causal Links:      {results["stats"]["graph"]}")


## 4. Inspect the Intelligence


In [ ]:
# 1. Check the Top Paper
if results['papers']:
    p = results['papers'][0]
    title = p['title']
    content = p['content'][:200]
    print(f'Top Paper: {title}')
    print(f'Snippet: {content}...')

# 2. Check Discovered Causal Chains
if results['graph_links']:
    print('
Sample Causal Chains:')
    for link in results['graph_links'][:5]:
        n1 = link['node1']
        n2 = link['node2']
        rel = link['relation']
        print(f'   {n1} --[{rel}]--> {n2}')


## 5. Construct the Expert Prompt


In [ ]:
prompt = engine.format_prompt(results, query)
print('Prompt ready.')


## 6. Generate Answer with Gemini Flash


In [ ]:
from google import genai
from google.colab import userdata
from IPython.display import Markdown, display

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    client = genai.Client(api_key=GOOGLE_API_KEY)
    print('Gemini Client Configured')
except Exception as e:
    print('Error: API Key not found.')

print('Generating response...')
try:
    response = client.models.generate_content(
        model='gemini-flash-latest',
        contents=prompt
    )
    display(Markdown('### Response:'))
    display(Markdown(response.text))
except Exception as e:
    print(f'Error: {e}')


## 7. Advanced: Build Your Own Prompt Template


In [ ]:
my_role = "You are a Data Journalist."
my_instruction = "Summarize in 3 bullet points."
papers_text = ""
for p in results['papers']:
    t = p['title']
    c = p['content'][:200]
    papers_text += f'- {t}: {c}...
'
graph_text = ""
for link in results['graph_links']:
    n1 = link['node1']
    n2 = link['node2']
    graph_text += f'- {n1} causes {n2}
'
custom_prompt = f"ROLE: {my_role}
QUESTION: {query}

DATA:
{papers_text}
{graph_text}

DO: {my_instruction}"
print("Custom prompt created.")


## 8. Bonus: Swap the Brain


In [ ]:
# 1. Download Test Brain
!wget -q https://github.com/nunezmatias/grafoRag/raw/main/examples/test_brain.zip
!unzip -o -q test_brain.zip

# 2. Initialize a NEW Engine
solar_engine = GraphRAGEngine(
    vector_db_path='./test_brain/test_db',
    graph_json_path='./test_brain/test_skeleton.json'
)

# 3. Run Query
solar_query = 'How does solar energy affect the grid stability?'
solar_results = solar_engine.search(solar_query, top_k=2)
print(f'Query: {solar_query}')
papers_count = len(solar_results['papers'])
links_count = len(solar_results['graph_links'])
print(f'Found {papers_count} papers and {links_count} links.')
if solar_results['graph_links']:
    link = solar_results['graph_links'][0]
    n1 = link['node1']
    n2 = link['node2']
    rel = link['relation']
    print(f'Link Found: {n1} --[{rel}]--> {n2}')


## 9. Advanced: Load from Google Drive


In [ ]:
MY_CUSTOM_GDRIVE_ID = "1iKcEzECN9LTMi3bIq4ocRfFJgvb1dLus"
try:
    drive_engine = GraphRAGEngine(gdrive_id=MY_CUSTOM_GDRIVE_ID)
    print('Custom Brain Loaded')
    solar_query = 'How does solar energy affect the grid stability?'
    drive_results = drive_engine.search(solar_query, top_k=2)
    print(f'Query: {solar_query}')
    papers_count = len(drive_results['papers'])
    links_count = len(drive_results['graph_links'])
    print(f'Found {papers_count} papers and {links_count} links.')
    if drive_results['graph_links']:
        link = drive_results['graph_links'][0]
        n1 = link['node1']
        n2 = link['node2']
        rel = link['relation']
        print(f'Link Found: {n1} --[{rel}]--> {n2}')
except Exception as e:
    print(f'Error: {e}')
